# Ultimate Guitar Scraping. First part: extracting filter items

Let's create a Selenium Driver to access the page

In [181]:
import requests
from selenium import webdriver
import os
import re
import codecs
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.options import Options 

HTML_PATH = '/Users/xf20939/Desktop/HALLELUJAH CHORDS (ver 2) by Jeff Buckley @ Ultimate-Guitar.Com.html'
#CHORD_CLASS_NAME = '_3PpPJ OrSDI'
URL = 'https://tabs.ultimate-guitar.com/tab/ed-sheeran/perfect-chords-1956589'



TODOs:

1) change my function to populate a dataframe given a LIST
2) instead of giving me chord list, make jack's code give me out a dataframe

In [182]:
#generate a list of URLs leading to the various mainpages. Mainpages contain lists of chord pages.


In [183]:

def generate_list_of_chord_page_links(current_main_page):
    #takes in a link containing a list of pages, finds all links to chord pages, returns list of these links.
    CHORD_PAGE_LINK_CLASSNAME = '_3DU-x JoRLr _3dYeW'
    main_soup = generateSoupOfWebpage(current_main_page)
    links_to_chord_page = main_soup.find_all('a', { 'class' : CHORD_PAGE_LINK_CLASSNAME}, href=True)
    list_of_chord_page_links = []
    for chord_page in links_to_chord_page:
        list_of_chord_page_links.append(chord_page['href'])
    return list_of_chord_page_links



In [184]:
                 

def generateSoupOfWebpage(url):   
    #takes in a URL, returns its soup
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument('--headless')
    driver = webdriver.Chrome("./chromedriver", options=options)
    driver.get(url)
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    driver.close()

    return BeautifulSoup(html_source_code, 'html.parser')


def generateSoupForSetOfWebpages(url_list):
    #takes in a list of urls. returns a list of page soups.
    soups = []    
    for url in url_list:
        soups.append(generateSoupOfWebpage(url))  
    return soups



In [72]:

def getSongInfo(page_soup):
    #takes in the soup of HTML and the class name for the HTML class in which the song info is kept
    INFO_CLASS_NAME = '_3rlxz'
    song_info_spans = page_soup.find_all('span', { 'class' : INFO_CLASS_NAME }) #finds span objects with the correct class
    song_info = [str(i.contents[0]) for i in song_info_spans] #finds contents of the span objects
    for stri in song_info:
        if '<span' in stri:
            song_info.remove(stri) #removes some excess spans that weirdly ended up in there... check on other songs
    #' '.join(song_info)
    return song_info

def extractChordsFromPageSoup(page_soup):
    #Given a specific path to a HTML file, convert the HTML into a list of chords. Function takes in the BeautifulSoup object for a chord page, and the class name of spans in which chords are kept.
    CHORD_CLASS_NAME = '_3PpPJ OrSDI' # this can be changed if UltimateGuitar tweak the site
    spans_containg_chords = page_soup.find_all('span', { 'class' : CHORD_CLASS_NAME })
    song_chords = [i.contents[0] for i in spans_containg_chords]
    return song_chords

def append_to_df(global_dataframe, title, author, song_information, chord_information):
    #takes in the global data frame, and all the information needed to populate the rows of the df. appends this information on to the global dataframe and returns the dataframe.
    local_dataframe = pd.DataFrame()
    local_dataframe['title'] = [title]
    local_dataframe['author'] = [author]
    local_dataframe['song_information'] = [' '.join(song_information)]
    local_dataframe['chords'] = [chord_information]
    return global_dataframe.append(local_dataframe)
    
    

In [185]:
def build_large_df(chord_page_soups):
    #takes in the url of the main page of chord page links. e.g page 6 of the large list of chord links. returns data frame of all songs in there.
    #list_of_chord_pages = generate_list_of_chord_page_links(main_page_url)
    #soups = generateSoupForSetOfWebpages(list_of_chord_pages)
    all_chords = pd.DataFrame()
    for soup in chord_page_soups:
        try:
            title = soup.find('h1').get_text()
            author = soup.find_all('div', { 'class' : "_-8l1v" })[0].get_text()
            chords = extractChordsFromPageSoup(soup)
            song_info = getSongInfo(soup)
            all_chords = append_to_df(all_chords, title, author, song_info, chords)
        except:
            print('chord extraction problem')
            pass
        
    return all_chords
        #append it to the dataframe




In [190]:
#create the list of main urls by filling in the page number. do 1 - 10, then 11 - 20, etc.
eleven_20 = [i for i in range(71,81)]
main_url_list = []
for page in eleven_20:
    string_in_string = "https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page={}&type[]=Chords".format(page)
    main_url_list.append(str(string_in_string))


In [191]:
main_url_list

['https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=71&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=72&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=73&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=74&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=75&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=76&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=77&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=78&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=79&type[]=Chords',
 'https://www.ultimate-guitar.com/explore?order=hitstotal_desc&page=80&type[]=Chords']

In [192]:
#MAKE A BIG LIST OF SONG PAGES TO SCRAPE
chord_page_links_accumulator = []
for main_url in main_url_list:
    #get the two dfs and append together
    #first make the soup of the list of chord pages
    chord_page_links_accumulator.append(generate_list_of_chord_page_links(main_url))

#just flattens the list
all_chord_page_links = [item for sublist in chord_page_links_accumulator for item in sublist]

#double check this works with a list of 2 then proceed
soups = generateSoupForSetOfWebpages(all_chord_page_links)
new_global_df = build_large_df(soups)
new_global_df.to_csv("./scraped_chords_ultimateguitar/71_to_80.csv") 

chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem
chord extraction problem


In [193]:
print('\a')


